In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_path = '../Data/BestPlayers.csv'

In [ ]:
df = pd.read_csv(file_path, sep=';', encoding='utf-8')
print(df.shape)
df.head()

In [ ]:
def player_top_game(frame):
    i = frame['Elo'].idxmax()
    res = frame.loc[i, ['Date','PlayerName', 'Elo']]
    return res

In [ ]:
def top_players(frame):
    player_games = frame.groupby('PlayerName')
    top_player_scores = player_games.apply(player_top_game) 
    top_sorted = top_player_scores.sort_values('Elo', ascending=False)
    ten_top =  top_sorted[:10]   
    return ten_top

In [ ]:
df['Date'] = df['Date'].map(lambda x: int(x[:4]))

In [ ]:
player_elo_df_na = pd.DataFrame(np.concatenate([df.loc[:, ['Date', 'WhiteName', 'WhiteElo']], df.loc[:, ['Date', 'BlackName', 'BlackElo']]]))
player_elo_df_na.columns = ['Date', 'PlayerName', 'Elo']
player_elo_df = player_elo_df_na.dropna()
player_elo_df['Elo'] = player_elo_df['Elo'].astype(int)
player_elo_df = player_elo_df.sort_values('Date')

players_by_year = player_elo_df.groupby('Date')
top_players_by_year = players_by_year.apply(top_players)
top_players_by_year.drop_duplicates()

In [ ]:
best_players_path = '../Data/TopPlayersPerYear.csv'
top_players_by_year.to_csv(best_players_path, sep=';', index=False, encoding='utf-8', chunksize=1000)

In [ ]:
players = top_players_by_year['PlayerName']
players = players.drop_duplicates()

In [ ]:
def get_player_careers(frame):
    years = frame.groupby('Date')
    top_elo_by_year = years.apply(player_top_game)
    player_career = top_elo_by_year[top_elo_by_year['PlayerName'].isin(players)]
    return player_career

In [ ]:
games_of_top_players = player_elo_df.groupby('PlayerName').apply(get_player_careers)

In [ ]:
careers_path = '../Data/TopPlayerCareers.csv'
games_of_top_players.to_csv(careers_path, sep=';', index=False, encoding='utf-8', chunksize=1000)